In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sys

print("Python 环境路径:", sys.prefix)
print("Python 版本:", sys.version)
print("Python 可执行文件路径:", sys.executable)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
#扩展维度
train_images = np.expand_dims(train_images,-1)
test_images = np.expand_dims(test_images,-1)

In [ ]:
train_images.shape

In [ ]:
#建立模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64,(3,3),
                                 input_shape=train_images.shape[1:],
                                 activation='relu'),)
model.add(tf.keras.layers.MaxPooling2D())#默认2x2
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())#全局平均池化
model.add(tf.keras.layers.Dense(10,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

In [ ]:
#模型训练
history=model.fit(train_images,
                  train_labels,
                  epochs=5,
                 validation_data=(test_images,test_labels))

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.epoch,history.history.get('acc'),label='acc')
plt.plot(history.epoch,history.history.get('val_acc'),label='val_acc')
plt.legend()

In [ ]:
#模型保存
# tf.saved_model.save(model,"./fashion_mnist/v1")

**------------------------------------------------------------------------------------**

### 加载模型，并读取签名

In [ ]:
new_model=tf.saved_model.load("./fashion_mnist/v1")
model_signature=new_model.signatures["serving_default"]
print(model_signature)

### 查看模型输入输出

In [ ]:
model_signature.inputs,model_signature.outputs

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()
print(test_images.shape)

In [ ]:
x_test=tf.keras.utils.normalize(test_images,axis=1)#归一化处理
# x_test=tf.cast(x_test,dtype=tf.float32)#更改数据类型
# print(x_test.shape)
x_test = np.expand_dims(x_test,-1)#扩展维度
print(x_test.shape)

In [ ]:
x_test=tf.data.Dataset.from_tensor_slices(x_test)#将数据加载为tensorspec
print("shape1: ",x_test)
x_test=x_test.shuffle(100)
# x_test=x_test.batch(2)
# print("shape2: ",x_test)
# print("len: ",len(x_test))

### 通过take方法读取一个数据

In [ ]:
batch=next(iter(x_test.take(1)))#通过take方法从batch中获取一个数据
plt.imshow(tf.keras.preprocessing.image.array_to_img(batch))#显示数据，通过array_to_img将数组转化为图像
dataset=tf.constant(batch)#将数组转化为tensor
print("shape1: ",dataset.shape)
dataset=np.expand_dims(dataset,0)#扩展维度
print("shape2: ",dataset.shape)

### 通过next方式读取一个数据

In [ ]:
batch=next(iter(x_test))
plt.imshow(tf.keras.preprocessing.image.array_to_img(batch))
batch = next(iter(x_test))#迭代器，取出一个batch的数据
dataset=tf.constant(batch)#转化为tensor对象
print("shape: ",dataset.shape)
dataset=np.expand_dims(dataset,0)
print("shape2: ",dataset.shape)

### 模型测试

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))


In [ ]:
resout=model_signature(conv2d_input=dataset)

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
def plot_value_array(predictions_array):
    plt.grid(False)
    plt.xticks(range(10),class_names,rotation=45)
    plt.yticks()
    plt.bar(range(10),predictions_array['dense'][0])
    plt.ylim([0,1])

In [ ]:
plot_value_array(resout)